In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# 시뮬레이션 파라미터
dt = 0.1          # 타임 스텝 [초]
T = 40           # 전체 시뮬레이션 시간 [초]
N_steps = int(T / dt)
N_pn = 4          # PN 비례 게인

####DRAG
import random

r1 = random.random()
r2 = random.random() 

v1 = random.random()
v2 = random.random()

v1_norm = v1/(v1**2 + v2**2)**0.5
v2_norm = v2/(v1**2 + v2**2)**0.5

# 초기 위치 및 속도
target_pos = np.array([ 10.0, 0.0])
target_vel = np.array([ -10*v1_norm , -10*v2_norm ])  # 목표 속도
missile_pos = np.array([-300.0, -300.0])
missile_speed = 20.0  # 미사일 속도 (2배 빠름)

# 기록 저장
target_traj = []
missile_traj = []

In [141]:

# 초기 LOS angle
def los_angle(missile, target):
    vec = target - missile
    return np.arctan2(vec[1], vec[0])

prev_lambda = los_angle(missile_pos, target_pos)


In [142]:

# 시뮬레이션 루프
for step in range(N_steps):
    time = step * dt

    # 목표 랜덤 움직임 (랜덤 가속, 부드러운)
    acc_noise = np.random.uniform(-2, 2, size=2)
    target_vel += acc_noise * dt
    target_pos += target_vel * dt

    # LOS angle & 변화율 계산
    lambda_now = los_angle(missile_pos, target_pos)
    lambda_dot = (lambda_now - prev_lambda) / dt
    prev_lambda = lambda_now

    # 폐쇄 속도
    rel_vel = target_vel - missile_speed * np.array([
        np.cos(lambda_now), np.sin(lambda_now)
    ])
    closing_vel = -np.dot(rel_vel, (target_pos - missile_pos) / np.linalg.norm(target_pos - missile_pos))

    # PN 가속 방향
    accel_mag = N_pn * closing_vel * lambda_dot
    accel_dir = np.array([-np.sin(lambda_now), np.cos(lambda_now)])  # 수직 방향
    missile_accel = accel_mag * accel_dir

    # 미사일 속도 업데이트 (단위 벡터 방향 고정, 속도 일정)
    missile_vel = missile_speed * np.array([
        np.cos(lambda_now), np.sin(lambda_now)
    ])
    missile_pos += missile_vel * dt

    # 저장
    target_traj.append(target_pos.copy())
    missile_traj.append(missile_pos.copy())

    # 요격 여부 검사 (근접)
    if np.linalg.norm(missile_pos - target_pos) < 5:
        print(f"타격 성공! Time: {time:.2f} s")
        break


타격 성공! Time: 15.90 s


In [143]:
from matplotlib.animation import FFMpegWriter

writer = FFMpegWriter(fps=20, metadata=dict(artist='Me'), bitrate=1800)

In [144]:
# trajectory를 numpy 배열로 변환 (매우 중요!!)
target_traj = np.array(target_traj)
missile_traj = np.array(missile_traj)

N = min(len(target_traj), len(missile_traj))

fig, ax = plt.subplots()
ax.set_xlim(-300, 300)
ax.set_ylim(-300, 300)
ax.set_aspect('equal')

target_dot, = ax.plot([], [], 'ro', label='Target')
missile_dot, = ax.plot([], [], 'bo', label='Missile')
traj_line1, = ax.plot([], [], 'r--', linewidth=1)
traj_line2, = ax.plot([], [], 'b--', linewidth=1)
ax.legend()

def update(frame):
    if frame >= N:
        frame = N - 1
    
    x1, y1 = target_traj[frame]
    x2, y2 = missile_traj[frame]

    target_dot.set_data([x1], [y1])
    missile_dot.set_data([x2], [y2])
    traj_line1.set_data(target_traj[:frame+1, 0], target_traj[:frame+1, 1])
    traj_line2.set_data(missile_traj[:frame+1, 0], missile_traj[:frame+1, 1])
    
    return target_dot, missile_dot, traj_line1, traj_line2

from matplotlib.animation import FuncAnimation, FFMpegWriter

ani = FuncAnimation(fig, update, frames=N, interval=50, repeat=False)

try:
    writer = FFMpegWriter(fps=20, metadata=dict(artist='Me'), bitrate=1800)
    ani.save("missile_simulation.mp4", writer=writer)
    print("✅ 저장 성공: missile_simulation.mp4")
except Exception as e:
    print("🚨 저장 실패:", e)

plt.close(fig)


✅ 저장 성공: missile_simulation.mp4
